In [11]:
import torch

In [1]:
from src.model import get_model
from src.dataset import FrameDataset, split_dataset, get_pandas_dataset

/Users/alexgiving/Documents/PythonProjects/Traffic_Object_Detection_Lab3/venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [77]:
baseTrainPath = 'data/train_images/train_images/'
dataset_path = 'data/usdc_train.json'

In [106]:
pandas_dataset, classes = get_pandas_dataset(dataset_path)

frame_dataset = FrameDataset(pandas_dataset, baseTrainPath)
train_dataset, test_dataset = split_dataset(frame_dataset)

pin_memory = torch.cuda.is_available()

train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=16,
    shuffle=True,
    collate_fn=lambda x: x,
    pin_memory=pin_memory)

test_dataloader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=16,
    collate_fn=lambda x: x,
    pin_memory=pin_memory)

In [90]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

device(type='cuda')

In [ ]:
model = get_model(len(classes))

In [91]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=0.0005)
num_epochs = 5

In [107]:
model.to(device)

for epoch in range(num_epochs):
    epoch_loss = 0
    for data in train_dataloader:
        imgs = []
        targets = []
        for d in data:
            imgs.append(d[0].to(device))
            targ = {}
            targ['boxes'] = d[1]['boxes'].to(device)
            targ['labels'] = d[1]['labels'].to(device)
            targets.append(targ)
        loss_dict = model(imgs, targets)
        loss = sum(v for v in loss_dict.values())
        epoch_loss += loss.cpu().detach().numpy()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(epoch_loss)

KeyboardInterrupt: 